In [1]:
!gpustat -f

gpu22                Tue Jun  4 21:42:31 2024  550.54.15
[0] NVIDIA RTX A6000 | 83°C,  34 % |  7418 / 49140 MB | wangyi(7410M)
 └─ 1084644 (  90%, 5222MB): python /home/lr/wangyi/EMNLP2024/MMT/baseline/VGAMT/main.py --beam_size 4 --exp_name VGAMTtestDE --dump_path /raid_elmo/home/lr/wangyi/temp --features_path /home/lr/wangyi/EMNLP2024/MMT/baseline/VGAMT/data/multi30kAttribute/features/mdetr_features,/home/lr/wangyi/EMNLP2024/MMT/baseline/VGAMT/data/multi30kAttribute/features/clip_features --features_type mdetr+clip --data_path /home/lr/wangyi/EMNLP2024/MMT/baseline/VGAMT/data/multi30kNofilter --data_mix_path /home/lr/wangyi/EMNLP2024/MMT/baseline/VGAMT/data/multi30kNofilter --features_mix_path /home/lr/wangyi/EMNLP2024/MMT/baseline/VGAMT/data/multi30kAttribute/features/mdetr_features,/home/lr/wangyi/EMNLP2024/MMT/baseline/VGAMT/data/multi30kAttribute/features/clip_features --src_lang en --tgt_lang de --dropout 0.4 --batch_size 32 --optimizer adam_inverse_sqrt,lr=0.00001,warmup_updates

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [2]:
from transformers import (
    AutoTokenizer,
    LEDForConditionalGeneration,
    AutoModelForSeq2SeqLM,
)
from datasets import load_dataset, load_metric
import torch
import nltk
from tqdm.notebook import tqdm

rouge = load_metric("rouge")

/home/lr/tanshiyin/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_4013361/3356187376.py:11: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")
/home/lr/tanshiyin/anaconda3/envs/mds/lib/python3.9/site-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to 

In [3]:
if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print(f"发现 {device_count} 个可用的 GPU:")
    for i in range(device_count):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
    device = torch.device('cuda')
else:
    print("未发现可用的 GPU")
    device = torch.device('cpu')

发现 1 个可用的 GPU:
GPU 0: NVIDIA RTX A6000


In [4]:
dataset=load_dataset('multi_news')

/home/lr/tanshiyin/anaconda3/envs/mds/lib/python3.9/site-packages/datasets/load.py:1429: FutureWarning: The repository for multi_news contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/multi_news
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [55]:
num_items = 10
item_embeddings = torch.rand(num_items, 4)
item_embeddings_static = item_embeddings.clone()
predicted_item_embedding = torch.rand(2,4)


In [65]:
for item_id_idx, item_id in enumerate(range(num_items)):
    print(item_id, item_id_idx)

0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9


In [5]:
import random
data_idx = random.choices(range(len(dataset['test'])),k=10)
# data_idx = range(len(dataset['test']))
dataset_small = dataset['test'].select(data_idx)

In [164]:
for doc_target in dataset_small:
    docs = doc_target['document']
    target = doc_target['summary']
    
    all_docs = docs.split("|||||")
    for i, doc in enumerate(all_docs):
#         doc = doc.replace("\n", " ")
#         doc = " ".join(doc.split())
        all_docs[i] = doc
        if doc.find("Associated Press") != -1:
            print("字符串中包含 Associated Press")
        if doc.find("FILE -") != -1:
            print("字符串中包含 FILE -")
#         else:
#             print("字符串中不包含 Associated Press")
    
#     find("(Associated Press)")
    
    score=rouge.compute(predictions=all_docs[0], references=target, use_stemmer=True)

字符串中包含 Associated Press
字符串中包含 FILE -


ValueError: Mismatch in the number of predictions (36) and references (1698)

### Document和target的相似度

In [115]:
doc_idx = 0

docs = dataset['test'][doc_idx]['document']
target = dataset['test'][doc_idx]['summary']

all_docs = docs.split("|||||")
for i, doc in enumerate(all_docs):
    doc = doc.replace("\n", " ")
    doc = " ".join(doc.split())
    all_docs[i] = doc

num_docs = len(all_docs)

In [135]:
num_docs = len(all_docs)

In [76]:
# Jaccard 相似度计算target和docs之间的相似度

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import jaccard_score


# 将文本转换为n-gram集合
vectorizer = CountVectorizer().fit_transform(all_docs+[target])
vectors = vectorizer.toarray()

for i in range(num_docs):
    vectors[i][vectors[i]>0] = 1
vectors[-1][vectors[-1]>0] = 1

# 计算Jaccard相似度
for i in range(num_docs):
    jaccard_sim = jaccard_score(vectors[i], vectors[-1], average='binary')
    print("Jaccard Similarity:", jaccard_sim)

Jaccard Similarity: 0.16864608076009502
Jaccard Similarity: 0.17391304347826086


In [77]:
# TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# 将文本向量化
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(all_docs+[target])

# 计算余弦相似度
for i in range(num_docs):
    cosine_sim = cosine_similarity(tfidf_matrix[i], tfidf_matrix[-1])
    print("Cosine Similarity:", cosine_sim[0][0])


Cosine Similarity: 0.6288669160994215
Cosine Similarity: 0.6330491122822253


In [78]:
# 编辑距离（Levenshtein距离）

import Levenshtein


# 计算Levenshtein距离
for i in range(num_docs):
    levenshtein_dist = Levenshtein.distance(all_docs[i], target)
    print("Levenshtein Distance:", levenshtein_dist)

Levenshtein Distance: 4395
Levenshtein Distance: 4356


In [79]:
# BLEU分数

from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu

# 示例文本
reference = "The cat sat on the mat".split()
candidate = "The cat is on the mat".split()

# 使用平滑函数，平滑函数可以处理没有重叠的情况，通过对0计数的n-gram添加一个小的平滑值来避免分母为0。
smooth = SmoothingFunction()

for i in range(num_docs):
    reference = all_docs[i].split()
    candidate = target.split()
    # 计算BLEU分数
    bleu_score = sentence_bleu([reference], candidate, smoothing_function=smooth.method1)
    print("BLEU Score:", bleu_score)

BLEU Score: 0.0022353499841904303
BLEU Score: 0.0023272010157348983


In [80]:
# 句向量相似度 sentence transformer

from sentence_transformers import SentenceTransformer, util

# 加载预训练的句向量模型
model = SentenceTransformer('all-mpnet-base-v2')


# 计算文本的句向量
for i in range(num_docs):
    embedding1 = model.encode(all_docs[i], convert_to_tensor=True)
    embedding2 = model.encode(target, convert_to_tensor=True)

    # 计算余弦相似度
    cosine_sim = util.pytorch_cos_sim(embedding1, embedding2)
    print("Sentence Embedding Cosine Similarity:", cosine_sim.item())


Sentence Embedding Cosine Similarity: 0.8549716472625732
Sentence Embedding Cosine Similarity: 0.8432109355926514


### Document中的sentence和target的相似度

句子分割方式：
1. 直接用'.'进行分割，会把某些包含'.'的单词分割，不准确
2. 用nltk.sent_tokenize(text)方式，不会把单词分割，但是会把'T.I.'当作完整的句子，较准确
3. 使用spacy的模型分割，不会把'T.I.'当作完整句子，目前使用, 但是会把'Washington:'当作完整句子；尝试使用更大的模型：en_core_web_trf, en_core_web_lg，从https://spacy.io/models/en#en_core_web_sm
``` python
nlp = spacy.load("en_core_web_sm")
nlp.disable_pipe("parser")
nlp.enable_pipe("senter") # 这种设定更快，而且更准
# 处理文本
doc = nlp(text)
# 句子分割
sentences = [sent.text for sent in doc.sents]
```

In [15]:
import spacy

# 加载英文模型，用于句子分割
nlp = spacy.load("en_core_web_lg")

nlp.disable_pipe("parser")
nlp.enable_pipe("senter")

# import en_core_web_trf
# nlp = en_core_web_trf.load()

In [7]:
doc_idx = 0

docs = dataset['test'][doc_idx]['document']
target = dataset['test'][doc_idx]['summary']

all_docs = docs.split("|||||")
for i, doc in enumerate(all_docs):
#     doc = doc.replace("\n", " ")
#     doc = " ".join(doc.split())
    all_docs[i] = doc

num_docs = len(all_docs)

In [18]:
all_senteces = []
senteces_id_of_doc = []
for i in range(num_docs):
    doc = all_docs[i].replace("(Associated Press)", "").replace("FILE -", "")
    # 处理文本
    doc = nlp(doc)
    # 句子分割
    sentences = [" ".join(sent.text.replace('\n', ' ').split()) for sent in doc.sents]
    all_senteces.extend(sentences)
    if len(senteces_id_of_doc)==0:
        senteces_id_of_doc.append(len(sentences))
    else:
        senteces_id_of_doc.append(len(sentences)+senteces_id_of_doc[-1])

In [19]:
# Jaccard 相似度计算target和docs之间的相似度

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import jaccard_score


# all_senteces = []
# for i in range(num_docs):
#     sentences = nltk.sent_tokenize(all_docs[i])
#     all_senteces.extend(sentences)


    
# 将文本转换为n-gram集合
vectorizer = CountVectorizer().fit_transform(all_senteces+[target])
vectors = vectorizer.toarray()

for i in range(len(vectors)):
    vectors[i][vectors[i]>0] = 1

# 计算Jaccard相似度
doc_id = 0
for i in range(len(all_senteces)):
    
    if i == senteces_id_of_doc[doc_id]:
        doc_id += 1
    
    jaccard_sim = jaccard_score(vectors[i], vectors[-1], average='binary')
    print(f"doc {doc_id} sent {i}-Jaccard Similarity:", jaccard_sim)

doc 0 sent 0-Jaccard Similarity: 0.17054263565891473
doc 0 sent 1-Jaccard Similarity: 0.09009009009009009
doc 0 sent 2-Jaccard Similarity: 0.03418803418803419
doc 0 sent 3-Jaccard Similarity: 0.1282051282051282
doc 0 sent 4-Jaccard Similarity: 0.16521739130434782
doc 0 sent 5-Jaccard Similarity: 0.06779661016949153
doc 0 sent 6-Jaccard Similarity: 0.136
doc 0 sent 7-Jaccard Similarity: 0.061946902654867256
doc 0 sent 8-Jaccard Similarity: 0.04065040650406504
doc 0 sent 9-Jaccard Similarity: 0.12396694214876033
doc 0 sent 10-Jaccard Similarity: 0.049586776859504134
doc 0 sent 11-Jaccard Similarity: 0.05128205128205128
doc 0 sent 12-Jaccard Similarity: 0.06542056074766354
doc 0 sent 13-Jaccard Similarity: 0.1103448275862069
doc 0 sent 14-Jaccard Similarity: 0.09322033898305085
doc 0 sent 15-Jaccard Similarity: 0.02608695652173913
doc 0 sent 16-Jaccard Similarity: 0.04132231404958678
doc 0 sent 17-Jaccard Similarity: 0.058823529411764705
doc 0 sent 18-Jaccard Similarity: 0.045045045045045

In [20]:
# TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# all_senteces = []
# for i in range(num_docs):
#     sentences = nltk.sent_tokenize(all_docs[i])
#     all_senteces.extend(sentences)



# 将文本向量化
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(all_senteces+[target])

# 计算余弦相似度
doc_id = 0
for i in range(len(all_senteces)):
    
    if i == senteces_id_of_doc[doc_id]:
        doc_id += 1
    
    cosine_sim = cosine_similarity(tfidf_matrix[i], tfidf_matrix[-1])
    print(f"doc {doc_id} sent {i}-Cosine Similarity:", cosine_sim[0][0])


doc 0 sent 0-Cosine Similarity: 0.24102418098008926
doc 0 sent 1-Cosine Similarity: 0.23391266613631823
doc 0 sent 2-Cosine Similarity: 0.07539999655241314
doc 0 sent 3-Cosine Similarity: 0.24212850187007307
doc 0 sent 4-Cosine Similarity: 0.31950391731366007
doc 0 sent 5-Cosine Similarity: 0.1797630735667592
doc 0 sent 6-Cosine Similarity: 0.25847237481195035
doc 0 sent 7-Cosine Similarity: 0.16996419229512022
doc 0 sent 8-Cosine Similarity: 0.15341846023710742
doc 0 sent 9-Cosine Similarity: 0.3253832925427495
doc 0 sent 10-Cosine Similarity: 0.12432858888706294
doc 0 sent 11-Cosine Similarity: 0.16739494375604586
doc 0 sent 12-Cosine Similarity: 0.15355150202098913
doc 0 sent 13-Cosine Similarity: 0.19549630373574334
doc 0 sent 14-Cosine Similarity: 0.21216530513647738
doc 0 sent 15-Cosine Similarity: 0.11667598705079932
doc 0 sent 16-Cosine Similarity: 0.08548231652024595
doc 0 sent 17-Cosine Similarity: 0.13809644520701314
doc 0 sent 18-Cosine Similarity: 0.0828451291993201
doc 0 

In [21]:
# 编辑距离（Levenshtein距离）

import Levenshtein


# 计算Levenshtein距离
doc_id = 0
for i in range(len(all_senteces)):
    
    if i == senteces_id_of_doc[doc_id]:
        doc_id += 1
    
    levenshtein_dist = Levenshtein.distance(all_senteces[i], target)
    print(f"doc {doc_id} sent {i}-Levenshtein Distance:", levenshtein_dist)

doc 0 sent 0-Levenshtein Distance: 680
doc 0 sent 1-Levenshtein Distance: 789
doc 0 sent 2-Levenshtein Distance: 786
doc 0 sent 3-Levenshtein Distance: 734
doc 0 sent 4-Levenshtein Distance: 730
doc 0 sent 5-Levenshtein Distance: 749
doc 0 sent 6-Levenshtein Distance: 697
doc 0 sent 7-Levenshtein Distance: 792
doc 0 sent 8-Levenshtein Distance: 760
doc 0 sent 9-Levenshtein Distance: 708
doc 0 sent 10-Levenshtein Distance: 770
doc 0 sent 11-Levenshtein Distance: 766
doc 0 sent 12-Levenshtein Distance: 801
doc 0 sent 13-Levenshtein Distance: 661
doc 0 sent 14-Levenshtein Distance: 757
doc 0 sent 15-Levenshtein Distance: 797
doc 0 sent 16-Levenshtein Distance: 778
doc 0 sent 17-Levenshtein Distance: 769
doc 0 sent 18-Levenshtein Distance: 827
doc 0 sent 19-Levenshtein Distance: 762
doc 0 sent 20-Levenshtein Distance: 779
doc 0 sent 21-Levenshtein Distance: 785
doc 0 sent 22-Levenshtein Distance: 764
doc 0 sent 23-Levenshtein Distance: 821
doc 0 sent 24-Levenshtein Distance: 796
doc 0 sent

In [22]:
# BLEU分数

from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu

# 使用平滑函数，平滑函数可以处理没有重叠的情况，通过对0计数的n-gram添加一个小的平滑值来避免分母为0。
smooth = SmoothingFunction()



doc_id = 0
for i in range(len(all_senteces)):
    reference = all_senteces[i].split()
    candidate = target.split()
    
    if i == senteces_id_of_doc[doc_id]:
        doc_id += 1
    
    # 计算BLEU分数
    bleu_score = sentence_bleu([reference], candidate, smoothing_function=smooth.method1)
    print(f"doc {doc_id} sent {i}-BLEU Score:", bleu_score)

doc 0 sent 0-BLEU Score: 0.052516537134932935
doc 0 sent 1-BLEU Score: 0.008922792017840799
doc 0 sent 2-BLEU Score: 0.0017050434639569296
doc 0 sent 3-BLEU Score: 0.004071048670237826
doc 0 sent 4-BLEU Score: 0.024833537164633254
doc 0 sent 5-BLEU Score: 0.004147175988212835
doc 0 sent 6-BLEU Score: 0.006245303961616687
doc 0 sent 7-BLEU Score: 0.003205995862201587
doc 0 sent 8-BLEU Score: 0.0034873454219144554
doc 0 sent 9-BLEU Score: 0.049667074329266515
doc 0 sent 10-BLEU Score: 0.002027649818726467
doc 0 sent 11-BLEU Score: 0.003605727923410432
doc 0 sent 12-BLEU Score: 0.03773881547728682
doc 0 sent 13-BLEU Score: 0.0042193278398767265
doc 0 sent 14-BLEU Score: 0.004931851192351146
doc 0 sent 15-BLEU Score: 0.003205995862201587
doc 0 sent 16-BLEU Score: 0.0028216345864346703
doc 0 sent 17-BLEU Score: 0.007096035310665079
doc 0 sent 18-BLEU Score: 0.0018028639617052157
doc 0 sent 19-BLEU Score: 0.003355507926125869
doc 0 sent 20-BLEU Score: 0.0034873454219144554
doc 0 sent 21-BLEU

In [23]:
# 句向量相似度 sentence transformer

from sentence_transformers import SentenceTransformer, util

# 加载预训练的句向量模型
model = SentenceTransformer('all-mpnet-base-v2')

doc_id = 0
# 计算文本的句向量
for i in range(len(all_senteces)):
    embedding1 = model.encode(all_senteces[i], convert_to_tensor=True)
    embedding2 = model.encode(target, convert_to_tensor=True)
    
    if i == senteces_id_of_doc[doc_id]:
        doc_id += 1
    # 计算余弦相似度
    cosine_sim = util.pytorch_cos_sim(embedding1, embedding2)
    print(f"doc {doc_id} sent {i}-Sentence Embedding Cosine Similarity:", cosine_sim.item())


doc 0 sent 0-Sentence Embedding Cosine Similarity: 0.7516170144081116
doc 0 sent 1-Sentence Embedding Cosine Similarity: 0.6076449751853943
doc 0 sent 2-Sentence Embedding Cosine Similarity: 0.5492553114891052
doc 0 sent 3-Sentence Embedding Cosine Similarity: 0.5720685720443726
doc 0 sent 4-Sentence Embedding Cosine Similarity: 0.718743085861206
doc 0 sent 5-Sentence Embedding Cosine Similarity: 0.5036206841468811
doc 0 sent 6-Sentence Embedding Cosine Similarity: 0.6540898084640503
doc 0 sent 7-Sentence Embedding Cosine Similarity: 0.3216194808483124
doc 0 sent 8-Sentence Embedding Cosine Similarity: 0.4208720922470093
doc 0 sent 9-Sentence Embedding Cosine Similarity: 0.5073840022087097
doc 0 sent 10-Sentence Embedding Cosine Similarity: 0.2851841151714325
doc 0 sent 11-Sentence Embedding Cosine Similarity: 0.40695667266845703
doc 0 sent 12-Sentence Embedding Cosine Similarity: 0.6262562274932861
doc 0 sent 13-Sentence Embedding Cosine Similarity: 0.3923552930355072
doc 0 sent 14-Se